In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data loader

In [ ]:
folder_path='sample_data/Room_back_home-2024-12-29_19-50-26/'
folder_path_img = folder_path+'/Camera'

gyro = pd.read_csv(folder_path+'Gyroscope.csv')
gyro['time_diff_s'] = gyro['time'].diff().fillna(0) / 1e9
gyro['roll'] = (gyro['x']*gyro['time_diff_s']).cumsum()
gyro['pitch'] = (gyro['y']*gyro['time_diff_s']).cumsum()
gyro['yaw'] = (gyro['z']*gyro['time_diff_s']).cumsum()
gyro.drop(['x','y','z'],axis=1,inplace=True)

acc = pd.read_csv(folder_path+'/Accelerometer.csv')

motion = pd.merge(gyro, acc, on=['time','seconds_elapsed'],how='inner')
motion.rename(columns={'x': 'ax','y': 'ay','z': 'az'}, inplace=True)


image_filenames = sorted(os.listdir(folder_path_img))[1:-1]
image_timestamps = [float(filename.split('.')[0]) for filename in image_filenames] 

def synchronize_data(imu_data, image_timestamps):
    synchronized_data = []
    for img_time in image_timestamps:
        # Find closest IMU timestamp
        closest_imu_index = (np.abs(imu_data['time'] - img_time*1e6)).argmin()
        synchronized_data.append({
            'image_time': img_time,
            'imu_roll': imu_data.iloc[closest_imu_index]['roll'],
            'imu_pitch': imu_data.iloc[closest_imu_index]['pitch'],
            'imu_yaw': imu_data.iloc[closest_imu_index]['yaw'],
            'imu_ax': imu_data.iloc[closest_imu_index]['ax'],
            'imu_ay': imu_data.iloc[closest_imu_index]['ay'],
            'imu_az': imu_data.iloc[closest_imu_index]['az'],
            'image_path': os.path.join(folder_path_img, image_filenames[image_timestamps.index(img_time)])
        })
    return pd.DataFrame(synchronized_data)

synchronized_synchronized_df = synchronize_data(motion, image_timestamps)
synchronized_synchronized_df['time_diff_s'] = synchronized_synchronized_df['image_time'].diff().fillna(0) / 1e3

In [ ]:
synchronized_synchronized_df.head()

# extracting and matching of image features

In [ ]:
def load_image(path):
    return cv2.imread(path)

image1 = load_image(synchronized_synchronized_df.iloc[0]['image_path'])
image2 = load_image(synchronized_synchronized_df.iloc[1]['image_path'])

orb = cv2.ORB_create(nfeatures=1000)

keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
keypoints2, descriptors2 = orb.detectAndCompute(image2, None)
 
# Initialize the BFMatcher with default params
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
 
# Match descriptors between frame I and frame I+1
matches = bf.match(descriptors1, descriptors2)
 
# Sort matches by distance (best matches first)
matches = sorted(matches, key=lambda x: x.distance)[0:100]

In [ ]:
img_matches = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Convert BGR to RGB
img_matches_rgb = cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB)

# Display the image in the notebook
plt.figure(figsize=(20,10))
plt.imshow(img_matches_rgb)
plt.axis('off')
plt.show()

# calculate essential matrix from images

In [ ]:
K = np.array([[730, 0, 620], [0, 730, 360], [0, 0, 1]])  # Example values

# Extract matched keypoints
pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches])
pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches])
 
 
# Compute the Fundamental matrix using RANSAC
F, inliers = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

# Convert mask to boolean array
mask = inliers.ravel().astype(bool)
pts1_inliers = pts1[mask]
pts2_inliers = pts2[mask]
 
# Compute the Essential matrix using the camera's intrinsic parameters 
E = K.T @ F @ K
 
# Decompose the Essential matrix to get R and t
_, R, t, mask = cv2.recoverPose(E, pts1_inliers, pts2_inliers, K)

In [ ]:
R, t

In [ ]:
import numpy as np
a = np.zeros((4,1)) 

In [ ]:
a.shape